# Importing the Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms

# ResNet from Scratch

### Residual Block
This block address the problem of vanishing gradient that can occur in very deep neural networks during training. The ***Residual Block*** introduces a shortcut (skip connection) that skips one or more layers, allowing the input to be directly added to the output of the block.  
Note that in the `forward`, this is applieddirectly to the input `x`, and not to the output `out`.

![ResNet Architecture](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-09-25_at_10.26.40_AM_SAB79fQ.png)

In [2]:
# Residual Block
class ResidualBlock(nn.Module):    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

### ResNet

In [5]:
class ResNet(nn.Module):    
    def __init__(self, block, layers, num_classes):
        super().__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3),
                        nn.BatchNorm2d(self.in_channels),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.adpool = nn.AdaptiveAvgPool2d((2, 2))
        self.fc = nn.Linear(in_features=512, out_features=num_classes)
    
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                            nn.Conv2d(in_channels=self.in_channels, 
                                      out_channels=out_channels,
                                      kernel_size=1,
                                      stride=stride),
                            nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.adpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

In [6]:
resnet34 = ResNet(block=ResidualBlock, layers=[3, 4, 6, 3], num_classes=10)

In [7]:
resnet34

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [8]:
from torchinfo import summary
summary(resnet34, input_shape=(32, 3, 224, 224))

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       9,472
│    └─BatchNorm2d: 2-2                  128
│    └─ReLU: 2-3                         --
├─MaxPool2d: 1-2                         --
├─Sequential: 1-3                        --
│    └─ResidualBlock: 2-4                --
│    │    └─Sequential: 3-1              37,056
│    │    └─Sequential: 3-2              37,056
│    │    └─ReLU: 3-3                    --
│    └─ResidualBlock: 2-5                --
│    │    └─Sequential: 3-4              37,056
│    │    └─Sequential: 3-5              37,056
│    │    └─ReLU: 3-6                    --
│    └─ResidualBlock: 2-6                --
│    │    └─Sequential: 3-7              37,056
│    │    └─Sequential: 3-8              37,056
│    │    └─ReLU: 3-9                    --
├─Sequential: 1-4                        --
│    └─ResidualBlock: 2-7                --